# Goal - Create conversation with artifacts
- Artifacts are presented in the system message (along with their history eventually)
- Artifacts are referenced in the conversation by their nickname
- Artifacts can be edited by the AI by rewriting their content (need to use Claude so that I can present this with XML tags)
- Users can modify artifacts directly and the changes are reflected in the conversation
- Demo CRUD
  - User creates artifact by requesting it - user deletes artifact
  - User retrieves artifact by requesting it
- Demo relationship between artifacts
- Demo anchors in artifacts?

## Scenario
- realestate agent is gathering information about a property (might use tools to get more information)
- they talk about the property
- they make a template email - which generates a new artifact
- they edit the email (making references to other artifacts)
- they send it

## Unknown
- [ ] How do I have the assistant generate an artifact?


In [1]:
%load_ext autoreload
%autoreload 2
from conversation import Artifact, Tool, Conversation


In [2]:
def get_listing(address):
    artifact = Artifact(
        identifier="18bacG4a",
        type="application/json", 
        title="742 Maple Street Listing", 
        content="""\
{
    "address": "742 Maple Street",
    "city": "Cedar Rapids",
    "state": "IA",
    "zip": "52402",
    "price": 185000,
    "beds": 3,
    "baths": 2,
    "sqft": 1450,
    "lot_size": 0.25,
    "year_built": 1978,
    "description": "Charming ranch-style home in established neighborhood. Updated kitchen with new appliances. Finished basement, attached 2-car garage, fenced backyard with mature trees. Close to schools and shopping.",
    "features": [           
        "Central air",
        "Forced air heating",
        "Hardwood floors",
        "Updated kitchen",
        "Finished basement",
        "Attached garage",
        "Fenced yard"
    ],
}
""")
    return str(artifact)

def get_comparables(address):
    artifact = Artifact(
        identifier="3baf9f83", 
        type="application/json", 
        title="742 Maple Street Comparables", 
        content="""\
[
    {
        "address": "738 Maple Street",
        "city": "Cedar Rapids", 
        "state": "IA",
        "price": 179900,
        "beds": 3,
        "baths": 2,
        "sqft": 1400,
        "year_built": 1975,
        "last_sold": {
            "date": "2020-03-15",
            "price": 165000
        },
        "distance_miles": 0.1,
        "zestimate": 183000
    },
    {
        "address": "755 Oak Drive",
        "city": "Cedar Rapids",
        "state": "IA", 
        "price": 192000,
        "beds": 3,
        "baths": 2.5,
        "sqft": 1500,
        "year_built": 1980,
        "last_sold": {
            "date": "2021-08-01",
            "price": 180000
        },
        "distance_miles": 0.3,
        "zestimate": 195000
    },
    {
        "address": "729 Elm Court",
        "city": "Cedar Rapids",
        "state": "IA",
        "price": 187500,
        "beds": 3,
        "baths": 2,
        "sqft": 1425,
        "year_built": 1977,
        "last_sold": {
            "date": "2020-11-30",
            "price": 175000
        },
        "distance_miles": 0.2,
        "zestimate": 191000
    }
]
""")
    return str(artifact)

def get_email_template():
    artifact = Artifact(
        identifier="98acb34d", 
        type="text/plain", 
        title="Prospective Buyer Listing Email Template", 
        content="""
Dear {buyer_name},

I wanted to bring to your attention an exciting new listing at {address}, {city}, {state} that I believe would be perfect for you.

This beautiful {beds} bedroom, {baths} bathroom home offers {sqft} square feet of living space and was built in {year_built}. It is currently listed at ${price:,}, which represents excellent value for this desirable neighborhood.

Some key features that make this property stand out:
- Spacious layout with {beds} bedrooms
- {sqft} square feet of living space
- Well-maintained home built in {year_built}
- Current Zestimate: ${zestimate:,}

To give you some context about the local market, there are several comparable properties in the immediate vicinity:
- A similar {beds} bed/{baths} bath home just {distance_miles} miles away recently sold for ${last_sold[price]:,}
- Nearby properties range from ${price:,} to ${price:,} in this area
- Most homes in this neighborhood were built in the 1970s-1980s

Would you like to schedule a viewing of this property? I have several time slots available this week and would be happy to show you around.

Best regards,
Your Real Estate Agent
""")
    return str(artifact)

get_listing_schema = {
    "name": "get_listing",
    "description": "Get details about a specific property listing",
    "input_schema": {
        "type": "object",
        "properties": {
            "address": {
                "type": "string",
                "description": "The street address to look up"
            }
        },
        "required": ["address"]
    }
}

get_comparables_schema = {
    "name": "get_comparables",
    "description": "Get comparable property listings in the area",
    "input_schema": {
        "type": "object",
        "properties": {
            "address": {
                "type": "string",
                "description": "The street address to find comparables for"
            },
        },
        "required": ["address"]
    }
}

# TODO - fill in the comp blanks
get_email_template_schema = {
    "name": "get_email_template",
    "description": "Get an email template for sending property listings to prospective buyers",
    "input_schema": {
        "type": "object",
        "properties": {
            "buyer_name": {
                "type": "string",
                "description": "Name of the prospective buyer"
            },
            "listing": {
                "type": "object",
                "description": "Property listing details to include in the email"
            }
        },
        "required": ["buyer_name", "listing"]
    }
}


tools = [
    Tool(get_listing_schema, get_listing),
    Tool(get_comparables_schema, get_comparables),
    Tool(get_email_template_schema, get_email_template),
]

In [19]:
c = Conversation(tools=tools)

In [20]:
c.say("Find the listing for 742 Maple Street");


[TextBlock(text='Okay, let me look up the property listing for 742 Maple Street.', type='text'), ToolUseBlock(id='toolu_01LvFehcaPccgWcU3mjLq5Uv', input={'address': '742 Maple Street'}, name='get_listing', type='tool_use')]
Okay, let me look up the property listing for 742 Maple Street.
Here are the key details for the listing at <a href="#18bacG4a">742 Maple Street</a>:

- 3 bedroom, 2 bath ranch home 
- 1,450 square feet on a 0.25 acre lot
- Built in 1978
- Updated kitchen with new appliances
- Finished basement
- Attached 2-car garage
- Fenced backyard
- Listed for $185,000

The listing description highlights its charming style, great neighborhood location close to schools and shopping, and nice features like central air, hardwood floors, and mature trees in the fenced yard.

Let me know if you need any other details about this property listing!


In [38]:
def extract_text_content(messages):
    for message in messages:
        content = message['content']
        
        # If content is a string, yield it directly
        if isinstance(content, str):
            yield content
            continue
            
        # If content is a list, process each item
        if isinstance(content, list):
            for item in content:
                # Handle string items
                if isinstance(item, str):
                    yield item
                    continue
                    
                # Handle dict-like items (including TextBlock, ToolUseBlock)
                item_dict = item.dict() if hasattr(item, 'dict') else item
                
                # Extract text from TextBlock
                if item_dict.get('type') == 'text':
                    yield item_dict['text']
                    
                # Extract content from tool results
                elif item_dict.get('type') == 'tool_result':
                    yield item_dict['content']

def extract_artifacts(messages):
    artifacts = []
    
    # Extract text content from messages
    for text in extract_text_content(messages):
        # Find all artifact blocks using regex
        artifact_pattern = r'<artifact\s+identifier="([^"]+)"\s+type="([^"]+)"\s+title="([^"]+)">(.*?)</artifact>'
        matches = re.finditer(artifact_pattern, text, re.DOTALL)
        
        # Convert each match to an Artifact instance
        for match in matches:
            identifier = match.group(1)
            type = match.group(2)
            title = match.group(3)
            content = match.group(4).strip()
            
            artifact = Artifact(identifier, type, title, content)
            
            # If we've seen this identifier before, remove the old artifact
            existing_identifiers = [a.identifier for a in artifacts]
            index_of_existing = existing_identifiers.index(identifier) if identifier in existing_identifiers else None
            if index_of_existing is not None:
                artifacts.pop(index_of_existing)    
            artifacts.append(artifact)
    
    return artifacts

# Extract artifacts directly from conversation messages
artifacts = extract_artifacts(c.messages)
for artifact in artifacts:
    print(f"Found artifact: {artifact}")
    print(artifact.identifier)


Found artifact: <artifact identifier="18bacG4a" type="application/json" title="742 Maple Street Listing">
{
    "address": "742 Maple Street",
    "city": "Cedar Rapids",
    "state": "IA",
    "zip": "52402",
    "price": 185000,
    "beds": 3,
    "baths": 2,
    "sqft": 1450,
    "lot_size": 0.25,
    "year_built": 1978,
    "description": "Charming ranch-style home in established neighborhood. Updated kitchen with new appliances. Finished basement, attached 2-car garage, fenced backyard with mature trees. Close to schools and shopping.",
    "features": [           
        "Central air",
        "Forced air heating",
        "Hardwood floors",
        "Updated kitchen",
        "Finished basement",
        "Attached garage",
        "Fenced yard"
    ],
}
</artifact>
18bacG4a


In [39]:
# Create test messages with duplicate artifacts
test_messages = [
    {
        "role": "user",
        "content": """Here's a house listing:
<artifact identifier="18bacG4a" type="application/json" title="742 Maple Street Listing">
{
    "address": "742 Maple Street",
    "city": "Cedar Rapids", 
    "state": "IA",
    "price": 185000,
    "beds": 3,
    "baths": 2
}
</artifact>"""
    },
    {
        "role": "assistant", 
        "content": """I'll analyze that listing for you:
<artifact identifier="18bacG4a" type="application/json" title="742 Maple Street Listing">
{
    "address": "745 Maple Street (CORRECTED)",
    "city": "Cedar Rapids",
    "state": "IA", 
    "price": 185000,
    "beds": 3,
    "baths": 2
}
</artifact>

Based on the listing details..."""
    }
]

# Extract and print artifacts from test messages
artifacts = extract_artifacts(test_messages)
for artifact in artifacts:
    print(f"\nFound artifact: {artifact}")
    print(f"Identifier: {artifact.identifier}")



Found artifact: <artifact identifier="18bacG4a" type="application/json" title="742 Maple Street Listing">
{
    "address": "745 Maple Street (CORRECTED)",
    "city": "Cedar Rapids",
    "state": "IA", 
    "price": 185000,
    "beds": 3,
    "baths": 2
}
</artifact>
Identifier: 18bacG4a


In [18]:
text = """<artifact identifier=18bacG4a type=application/json title=742 Maple Street Listing>
{
    "address": "742 Maple Street",
    "city": "Cedar Rapids",
    "state": "IA",
    "zip": "52402",
    "price": 185000,
    "beds": 3,
    "baths": 2,
    "sqft": 1450,
    "lot_size": 0.25,
    "year_built": 1978,
    "description": "Charming ranch-style home in established neighborhood. Updated kitchen with new appliances. Finished basement, attached 2-car garage, fenced backyard with mature trees. Close to schools and shopping.",
    "features": [           
        "Central air",
        "Forced air heating",
        "Hardwood floors",
        "Updated kitchen",
        "Finished basement",
        "Attached garage",
        "Fenced yard"
    ],
}

</artifact>"""


# Find all artifact blocks using regex
artifact_pattern = r'<artifact[^>]*>(.*?)</artifact>'
matches = re.finditer(artifact_pattern, text, re.DOTALL)
for match in matches:
    print(match)


<re.Match object; span=(0, 761), match='<artifact identifier=18bacG4a type=application/js>


In [ ]:
c.say(");

In [ ]:
c.say(");

In [ ]:
c.say(");

In [ ]:
c.say(");

In [ ]:
c.say(");

In [ ]:
c.say(");

In [ ]:
c.say(");

In [ ]:
c.say(");

In [ ]:
c.say(");